<a href="https://colab.research.google.com/github/Bhakthi-Shetty7811/hiver-assignment/blob/main/Part_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part C: Mini-RAG for Knowledge Base Answering
**Candidate:** Bhakthi Shetty

This section demonstrates a lightweight retrieval-augmented generation (RAG) system using sentence embeddings. The system indexes uploaded knowledge-base articles, performs similarity-based search for a given query, and returns the most relevant excerpts.

The workflow includes:
- Uploading and embedding KB documents
- Retrieving relevant content using cosine similarity
- Generating an answer using extractive summarization from the top-ranked result
- Evaluating retrieval performance using Recall@3

Finally, two example queries are answered using this mini-RAG pipeline to showcase retrieval accuracy and output quality.

**Step 1: Install + Imports**

In [16]:
!pip install sentence-transformers

import os
import numpy as np

# We use "all-MiniLM-L6-v2" for fast, compact sentence embeddings.
from sentence_transformers import SentenceTransformer
import shutil

**Step 2: Prepare KB folder and upload files**

In [17]:
KB_FOLDER = "/content/kb_articles"
os.makedirs(KB_FOLDER, exist_ok=True)

# Upload files in Colab UI
from google.colab import files
uploaded = files.upload()

# Move uploaded files to KB folder
for filename in uploaded.keys():
    shutil.move(filename, os.path.join(KB_FOLDER, filename))

print("Files moved to:", KB_FOLDER)


Saving article1.txt to article1.txt
Saving article2.txt to article2.txt
Saving article3.txt to article3.txt
Saving article4.txt to article4.txt
Saving article5.txt to article5.txt
Saving article6.txt to article6.txt
Saving article7.txt to article7.txt
Saving article8.txt to article8.txt
Saving article9.txt to article9.txt
Saving article10.txt to article10.txt
Files moved to: /content/kb_articles


**Step 3: Load KB articles**

Normalization collapses whitespace and removes accidental newlines.

In [18]:
def load_kb_articles(kb_folder=KB_FOLDER):
    kb = {}
    for fname in os.listdir(kb_folder):
        if fname.lower().endswith(".txt"):
            full_path = os.path.join(kb_folder, fname)
            with open(full_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                # Minimal normalization
                text = " ".join(text.split())
                kb[fname] = text
    return kb

kb_articles = load_kb_articles()
print("Loaded KB Articles:", list(kb_articles.keys()))

Loaded KB Articles: ['article4.txt', 'article6.txt', 'article7.txt', 'article2.txt', 'article9.txt', 'article8.txt', 'article5.txt', 'article1.txt', 'article10.txt', 'article3.txt']


**Step 4: Load embeddings model and embed KB**

For long articles we may want to chunk before embedding covered in improvements.

In [19]:
model = SentenceTransformer("all-MiniLM-L6-v2")  # baseline, fast

# Create embeddings for each article
kb_embeddings = {}

for name, text in kb_articles.items():
    kb_embeddings[name] = model.encode(text)

**Step 5: Similarity function + search helper**

Similarities are in [-1,1]; typical MiniLM scores for relevant docs are often between ~0.4-0.85 depending on length and domain.

In [20]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def search_kb(query, top_k=3):
    """
    Returns top K most relevant KB articles for a query.
    """
    query_emb = model.encode(query)
    scores = []

    for name, emb in kb_embeddings.items():
        sim = cosine_similarity(query_emb, emb)
        scores.append((name, sim))

    # Sort by highest similarity
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:top_k]

# Quick test
print("\n--- Test Search ---")
example_query = "How do I configure SLAs?"
print(search_kb(example_query, top_k=3))


--- Test Search ---
[('article4.txt', np.float32(0.43817064)), ('article10.txt', np.float32(0.23702598)), ('article2.txt', np.float32(0.10805674))]


**Step 6: Simple answer generation (no LLM) + confidence heuristic**

This step generates a simple extractive answer using the highest-ranked retrieved article and assigns a heuristic confidence score.

In [21]:
def generate_answer_from_retrieval(query, top_k=3, excerpt_len=400):
    """
    1) Retrieve top_k articles
    2) Construct a short answer using the top article excerpt
    3) Provide a naive confidence score based on top similarity
    """
    results = search_kb(query, top_k=top_k)
    if not results:
        return {"query": query, "retrieved": [], "answer": "", "confidence": 0.0}

    # Build retrieved list with snippets
    retrieved = []
    for name, sim in results:
        text = kb_articles.get(name, "")
        snippet = text[:excerpt_len] + ("..." if len(text) > excerpt_len else "")
        retrieved.append({"file": name, "similarity": round(sim, 4), "snippet": snippet})

    # Use top article to form a simple answer (extractive + one-line synth)
    top_name, top_sim = results[0]
    top_text = kb_articles[top_name]
    # Very naive extractive answer: first paragraph or first 400 chars
    answer_excerpt = top_text.split("\n\n")[0].strip()
    if not answer_excerpt:
        answer_excerpt = top_text[:excerpt_len]

    # Confidence heuristic:
    # scale sim from [0.2, 0.9] -> [0.4, 0.95] and clamp; if sim very low, low confidence
    sim = top_sim
    if sim < 0.2:
        confidence = 0.25
    else:
        # linear scaling between 0.2 and 0.9
        confidence = 0.4 + (sim - 0.2) * (0.55 / 0.7)
        confidence = min(max(confidence, 0.0), 0.99)

    answer = {
        "query": query,
        "retrieved": retrieved,
        "answer": answer_excerpt,
        "confidence": round(confidence, 2)
    }
    return answer


**Step 7: Evaluation dataset + two required queries**

In [22]:
# Example evaluation dataset (if you want to programmatically test recall)
evaluation_data = [
    {"query": "How to setup forwarding rules?", "correct": "article1.txt"},
    {"query": "Steps to create workflows", "correct": "article2.txt"},
    {"query": "How can I merge customer emails?", "correct": "article3.txt"},
    {"query": "How to configure SLAs?", "correct": "article4.txt"},
]

# The two specific queries required by the assignment:
queries_to_run = [
    "How do I configure automations in Hiver?",
    "Why is CSAT not appearing?"
]


**Step 8: Run the two queries and show results**

In [23]:
for q in queries_to_run:
    result = generate_answer_from_retrieval(q, top_k=3)
    print("\n--- QUERY ---")
    print(q)
    print("\nRetrieved articles (top 3):")
    for item in result["retrieved"]:
        print(f"- {item['file']} (sim={item['similarity']})")
    print("\nGenerated answer (excerpt):")
    print(result["answer"])
    print("\nConfidence:", result["confidence"])



--- QUERY ---
How do I configure automations in Hiver?

Retrieved articles (top 3):
- article1.txt (sim=0.5839999914169312)
- article7.txt (sim=0.45399999618530273)
- article10.txt (sim=0.4068000018596649)

Generated answer (excerpt):
Title: How to Set Up Email Forwarding Rules in Hiver Summary: Learn how to automatically forward customer emails to specific teams. Steps: 1. Go to Hiver Admin Panel → Automations → Rules. 2. Click Create Rule. 3. Choose the condition (Subject, Sender, Keywords). 4. Select Forward To as the action. 5. Add team members or shared inboxes. 6. Save the rule. Notes: • Forwarding works only when automation is enabled for the inbox. • Rules run in the order listed.

Confidence: 0.7

--- QUERY ---
Why is CSAT not appearing?

Retrieved articles (top 3):
- article9.txt (sim=0.1565999984741211)
- article10.txt (sim=0.15629999339580536)
- article4.txt (sim=0.1185000017285347)

Generated answer (excerpt):
Title: Fixing Slow Loading or Performance Issues Summary: Perf

**Step 9: Simple Recall@3 evaluator**

In [25]:
def evaluate_recall_at_3():
    correct_count = 0
    total = len(evaluation_data)

    for item in evaluation_data:
        query = item["query"]
        correct_file = item["correct"]

        top3 = [x[0] for x in search_kb(query, top_k=3)]

        print(f"\nQuery: {query}")
        print("Top 3:", top3)
        print("Correct:", correct_file)

        if correct_file in top3:
            correct_count += 1

    recall_at_3 = correct_count / total
    print("\n=========================")
    print("RECALL @ 3 =", recall_at_3)
    print("=========================")

evaluate_recall_at_3()


Query: How to setup forwarding rules?
Top 3: ['article1.txt', 'article2.txt', 'article4.txt']
Correct: article1.txt

Query: Steps to create workflows
Top 3: ['article2.txt', 'article5.txt', 'article1.txt']
Correct: article2.txt

Query: How can I merge customer emails?
Top 3: ['article5.txt', 'article3.txt', 'article6.txt']
Correct: article3.txt

Query: How to configure SLAs?
Top 3: ['article4.txt', 'article10.txt', 'article2.txt']
Correct: article4.txt

RECALL @ 3 = 1.0


In [26]:
print("\n=== Evaluation Completed ===")



=== Evaluation Completed ===
